In [1]:
import pandas as pd
import re
import sqlite3
import nltk
from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.multiclass import OneVsRestClassifier
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report
from sklearn.preprocessing import MultiLabelBinarizer
import numpy as np
import pickle
import skops.io as sio
import joblib

# Загрузка стоп-слов
nltk.download('stopwords')
russian_stopwords = stopwords.words('russian')

# Загрузка данных из базы данных
conn = sqlite3.connect('yt_videos_for_train_labeled.db')
videos_df = pd.read_sql('SELECT * FROM videos', conn)

# Предобработка текста
def clean_text(text):
    text = re.sub(r'<.*?>', '', text)
    text = re.sub(r'\d+', '', text)
    text = re.sub(r'[^\w\s]', '', text)
    text = text.lower()
    return text

# Конкатенация названия видео и описания
videos_df['text'] = videos_df['video_title'].astype(str) + " " + videos_df['video_description'].astype(str)
videos_df['text'] = videos_df['text'].apply(clean_text)

X = videos_df['text']
y = videos_df['profession']

# Разделение данных на обучающую и тестовую выборки
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Yurik\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [3]:
from sklearn.metrics import precision_score, recall_score, f1_score

# Разделение данных на обучающую и тестовую выборки
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

real_classes = y_test

tfidf_vectorizer = TfidfVectorizer(stop_words=russian_stopwords)

# Создание конвейера с TF-IDF векторизацией и OneVsRestClassifier с логистической регрессией
pipeline = Pipeline([
    ('tfidf', tfidf_vectorizer),
    ('clf', OneVsRestClassifier(LogisticRegression(solver='liblinear')))
])

# Создаем MultiLabelBinarizer и обучаем его на метках
mlb = MultiLabelBinarizer()
y_train_mlb = mlb.fit_transform(y_train.str.split(','))
y_test_mlb = mlb.transform(y_test.str.split(','))

# Обучение модели
pipeline.fit(X_train, y_train_mlb)

# Предсказание вероятностей классов
probabilities = pipeline.predict_proba(X_test)

# Выбор индексов классов с максимальной вероятностью для каждого экземпляра
predicted_class_indices = np.argmax(probabilities, axis=1)

# Преобразование индексов в наименования классов
predicted_classes = mlb.classes_[predicted_class_indices]

# Предсказание классов для тестового набора
predictions = pipeline.predict(X_test)

# Выбор индекса класса, на котором вы хотите сосредоточиться
class_index = 0
target_class = mlb.classes_[class_index]

# Преобразование предсказанных вероятностей в бинарные метки
threshold = 0.05
predictions_binary = (probabilities[:, class_index] >= threshold).astype(int)

# Создание новых меток для этого класса
y_test_single_class = y_test_mlb[:, class_index]

# Преобразование y_test в одинарные метки, если они в мультилейбловом формате
# Это зависит от того, как устроены ваши данные
# Если y_test уже в формате одиночных меток, этот шаг можно пропустить
real_classes = y_test.str.split(',').apply(lambda labels: labels[0])

# Расчет общей точности
accuracy = np.mean(predicted_classes == real_classes)
print(f"Точность: {accuracy}")

# Расчет точности, полноты и F1-меры
precision = precision_score(real_classes, predicted_classes, average='weighted', labels=np.unique(predicted_classes))
recall = recall_score(real_classes, predicted_classes, average='weighted', labels=np.unique(predicted_classes))
f1 = f1_score(real_classes, predicted_classes, average='weighted', labels=np.unique(predicted_classes))

print(f"Точность (Precision): {precision}")
print(f"Полнота (Recall): {recall}")
print(f"F1-мера: {f1}")

# Сохранение модели
filename = 'text_model.sav'
sio.dump(pipeline, open(filename, 'wb'))

import zlib
import json
import pickle

def save_sklearn_model(model, filename):
    # Сохранение параметров модели в формате JSON
    model_params = model.get_params()
    params_json = json.dumps(model_params)
    # Конвертация JSON в байты
    params_bytes = params_json.encode('utf-8')
    # Сжатие байтов
    compressed_bytes = zlib.compress(params_bytes)
    # Запись сжатых байтов в файл
    with open(filename, 'wb') as file:
        file.write(compressed_bytes)

with open(filename, 'wb') as file:
        pickle.dump(pipeline, file)

# Сохранение TfidfVectorizer
with open('text_model_tfidf_vectorizer.pkl', 'wb') as file:
    pickle.dump(tfidf_vectorizer, file)

# Сохранение MultiLabelBinarizer
with open('text_model_mlb.pkl', 'wb') as file:
    pickle.dump(mlb, file)

Точность: 0.5912246598167176
Точность (Precision): 0.6524858337479683
Полнота (Recall): 0.6021779097723094
F1-мера: 0.5972121447522909


TypeError: Object of type TfidfVectorizer is not JSON serializable

In [3]:
# Расчет средней длины текста
average_length = videos_df['text'].apply(len).mean()

print(f"Средняя длина текста: {average_length}")


Средняя длина текста: 148.90004998889137


In [13]:
import tensorflow as tf
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np

print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

# Предобработка данных и векторизация
tfidf_vectorizer = TfidfVectorizer(stop_words=russian_stopwords)
X_tfidf = tfidf_vectorizer.fit_transform(X)
y_mlb = MultiLabelBinarizer().fit_transform(y)

# Разбиение данных на обучающую и тестовую выборки
X_train, X_test, y_train, y_test = train_test_split(X_tfidf, y_mlb, test_size=0.2, random_state=42)

def sparse_to_dense_generator(X, y, batch_size):
    num_samples = X.shape[0]
    for start_idx in range(0, num_samples, batch_size):
        end_idx = min(start_idx + batch_size, num_samples)
        X_batch = X[start_idx:end_idx].toarray()  # Преобразование в плотный формат
        y_batch = y[start_idx:end_idx]
        yield X_batch, y_batch

# Использование генератора для создания объектов tf.data.Dataset
batch_size = 32
train_dataset = tf.data.Dataset.from_generator(
    lambda: sparse_to_dense_generator(X_train, y_train, batch_size),
    output_types=(tf.float32, tf.int32),
    output_shapes=([None, X_train.shape[1]], [None, y_train.shape[1]])
).prefetch(tf.data.AUTOTUNE)

test_dataset = tf.data.Dataset.from_generator(
    lambda: sparse_to_dense_generator(X_test, y_test, batch_size),
    output_types=(tf.float32, tf.int32),
    output_shapes=([None, X_test.shape[1]], [None, y_test.shape[1]])
).prefetch(tf.data.AUTOTUNE)

# Создание модели нейронной сети
model = Sequential()
model.add(Dense(512, input_shape=(X_train.shape[1],), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(y_train.shape[1], activation='sigmoid'))

# Компиляция модели
model.compile(optimizer=Adam(learning_rate=1e-4), loss='binary_crossentropy', metrics=['accuracy'])


# Обучение модели
model.fit(train_dataset, epochs=10, validation_data=test_dataset)

# Оценка модели на тестовых данных
score = model.evaluate(test_dataset)

# Вывод результатов оценки
print(f'Test loss: {score[0]} / Test accuracy: {score[1]}')

Num GPUs Available:  0
Epoch 1/10
      1/Unknown - 1s 882ms/step - loss: 0.6930 - accuracy: 0.0625

KeyboardInterrupt: 

: 

In [8]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score

# Разбиение данных на обучающую и тестовую выборки
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Векторизация данных с помощью TfidfVectorizer
tfidf_vectorizer = TfidfVectorizer(stop_words=russian_stopwords)  # Используйте 'english' для английского текста
X_train_tfidf = tfidf_vectorizer.fit_transform(X_train)
X_test_tfidf = tfidf_vectorizer.transform(X_test)

# Преобразование разреженных матриц в плотные
X_train_dense = X_train_tfidf.toarray()
X_test_dense = X_test_tfidf.toarray()

# Создание и обучение модели LDA
lda = LinearDiscriminantAnalysis()
lda.fit(X_train_dense, y_train)

# Прогнозирование на тестовом наборе
y_pred = lda.predict(X_test_dense)

# Оценка точности
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy}')

MemoryError: Unable to allocate 486. MiB for an array with shape (772, 82443) and data type float64